In [ ]:
from matplotlib.image import imread
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sympy

In [ ]:
import scipy.fftpack as spfft
from scipy.linalg import hadamard
from sklearn.linear_model import Lasso

In [ ]:
alpha=1E-4
max_iter=1E3

In [ ]:
def squarify(image):
    org_height, org_width = image.shape
    new_dim = max(org_height, org_width)
    
    delta_h = new_dim - org_height
    delta_w = new_dim - org_width
    
    color = [0, 0, 0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    square_image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)

    return square_image

In [ ]:
def reconstruct_frame_hadamard(noisyframe, lasso):
    if noisyframe.shape[0] != noisyframe[1]:
        noisyframe = squarify(noisyframe)
        
    ny, nx = noisyframe.shape
    A = hadamard(nx*ny)
    flat_frame = frame.T.reshape(-1,1)
    
    idx_nonzero = np.where(flat_frame != 0)[0]
    
    # compressed sample
    b = flat_frame[idx_nonzero]
    # compression matrix @ transform matrix
    Ac = A[idx_nonzero,:]
    
    # LASSO optimization
    lasso.fit(Ac, b)
    
    # Reconstruct frame
    coeff = np.array(lasso.coef_)
    reconstructed_image = np.array(sympy.fwht(coeff)).astype(np.float32).reshape(width,height).T

    return reconstructed_image

In [ ]:
def reconstruct_frame_dct(frame, lasso):
    ny, nx = frame.shape
    A = np.kron(
        spfft.idct(np.identity(nx), norm='ortho', axis=0),
        spfft.idct(np.identity(ny), norm='ortho', axis=0)
        )

    flat_frame = frame.T.reshape(-1,1)
    idx_nonzero = np.where(flat_frame != 0)[0]

    # compressed sample
    b = flat_frame[idx_nonzero]
    # compression matrix @ transform matrix
    Ac = A[idx_nonzero,:]

    # LASSO optimization
    lasso.fit(Ac, b)

    # Reconstruct frame
    Xat = np.array(lasso.coef_).reshape(nx, ny).T # stack columns

    # Get the reconstructed frame
    Xa = idct2(Xat)

    return Xa

In [ ]:
image = imread('../dog.jpg')
bw_image = np.mean(image, -1); # Convert RGB to grayscale